In [3]:
import numpy as np

from manipulation import running_as_notebook

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    Parser,
    DiagramBuilder,
    MeshcatVisualizer,
    ModelVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Role,
    BodyIndex,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Simulator,
    StartMeshcat,
    RollPitchYaw,
)

from IPython.display import clear_output

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [47]:
robot_directives = """
directives:
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [10]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [1, 0, 0]
        rotation: !Rpy { deg: [0, 0, 0]}
- add_model:
    name: Schunk_Gripper
    file: package://robotics_final_project/drum_sticks.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: Schunk_Gripper::body_frame
    X_PC:
        translation: [0, 0, 0]
        rotation: !Rpy { deg: [0, 0, 0]}
"""

In [50]:
stiffness=20000
dissipation=2

drum_kit = f"""
<?xml version="1.0" ?>
<sdf version="1.7">
    <model name="drumkit">
    <static>true</static>
    <link name="snare">
    <collision name="snare">
        <pose>1 .1 .4 0 .14 0</pose>
        <geometry>
            <cylinder>
                <length>.18</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="snare">
        <pose>.1 .1 .4 0 .14 0</pose>
        <geometry>
            <cylinder>
                <length>.18</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>
    <link name="tom1">
    <collision name="tom1">
        <pose>-.2 .3 .6 0 .55 0</pose>
        <geometry>
            <cylinder>
                <length>.1</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="tom1">
        <pose>-.2 .3 .6 0 .55 0</pose>
        <geometry>
            <cylinder>
                <length>.15</length>
                <radius>0.14</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="tom2">
    <collision name="tom2">
        <pose>-.2 .3 .6 0 .55 0</pose>
        <geometry>
            <cylinder>
                <length>.1</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="tom2">
        <pose>-.2 .62 .6 0 .55 0</pose>
        <geometry>
            <cylinder>
                <length>.15</length>
                <radius>0.16</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="tom3">
    <collision name="tom3">
        <pose>.1 .9 .3 0 0 0</pose>
        <geometry>
            <cylinder>
                <length>.4</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="tom3">
        <pose>.1 .9 .3 0 0 0</pose>
        <geometry>
            <cylinder>
                <length>.4</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="hihat">
    <collision name="hihat">
        <pose>-.1 -.1 .6 0 0 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.2 .2 .02</radii>
            </ellipsoid>
        </geometry>

        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="hihat">
        <pose>.1 -.25 .6 0 0 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.2 .2 .02</radii>
            </ellipsoid>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="crash">
    <collision name="crash">
         <pose>-.4 .1 .85 0 .3 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.26 .26 .005</radii>
            </ellipsoid>
        </geometry>

        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="crash">
        <pose>-.4 .1 .85 0 .3 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.26 .26 .005</radii>
            </ellipsoid>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="ride">
    <collision name="ride">
        <pose>-.25 1.1 .65 .3 .3 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.3 .3 .005</radii>
            </ellipsoid>
        </geometry>

        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="ride">
        <pose>-.25 1.1 .65 .3 .3 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.3 .3 .005</radii>
            </ellipsoid>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="kick">
    <collision name="kick">
        <pose>-.3 .47 .25 0 1.57 0</pose>
        <geometry>
            <cylinder>
                <length>.4</length>
                <radius>0.25</radius>
            </cylinder>
        </geometry>

        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="kick">
        <pose>-.3 .47 .25 0 1.57 0</pose>
        <geometry>
            <cylinder>
                <length>.4</length>
                <radius>0.25</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    </model>
    </sdf>
"""

meshcat.Delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=0.00001
    )

parser = Parser(plant)
parser.package_map().Add("robotics_final_project", "./")
parser.AddModelsFromString(robot_directives, ".dmd.yaml")

instance = parser.AddModelsFromString(drum_kit, ".sdf")


plant.Finalize()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()

context = diagram.CreateDefaultContext()

simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5.0 if running_as_notebook else 0.1);

# visualizer.Run(loop_once=not running_as_notebook)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9b055803-8855-4ac7-95ae-1ed8fd517a9b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>